In [1]:
import os

import timm
import torch
from torch import nn
from torch.utils.data import DataLoader

/home/uceezl8/.conda/envs/amls_ii-final-uceezl8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from A.dataset import build_dataset, build_transform

In [3]:
NUM_FINETUNE_CLASSES = 5
batch_size = 260
learning_rate = 1e-3
epochs = 20
n_workers_per_device = 2
CUDA_VISIBLE_DEVICES = os.getenv("CUDA_VISIBLE_DEVICES", "0,1,2,3")
NUM_WORKERS = n_workers_per_device * len(CUDA_VISIBLE_DEVICES.split(","))

In [4]:
model = timm.create_model(
    "convnextv2_base", pretrained=True, num_classes=NUM_FINETUNE_CLASSES
)
pretrained_cfg = model.pretrained_cfg

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [6]:
if torch.cuda.device_count() > 1:
    print(f"Let's use {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

Let's use 4 GPUs!


In [7]:
train_transform, test_transform = build_transform(pretrained_cfg=pretrained_cfg)
train_ds, test_ds = build_dataset(
    "Datasets/train.csv",
    "Datasets/train_images",
    allowed_labels=[0, 4],
    train_transform=train_transform,
    test_transform=test_transform,
)
train_dataloader = DataLoader(train_ds, batch_size=batch_size, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(test_ds, batch_size=batch_size, num_workers=NUM_WORKERS)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    from tqdm import tqdm

    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    with tqdm(total=len(dataloader.dataset)) as pbar:
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            # Compute prediction and loss
            pred = model(X)
            loss = loss_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            pbar.update(batch_size)
            # loss, current = loss.item(), batch * batch_size + len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [9]:
model.to(device)
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------


3380it [00:24, 139.35it/s]                          


Test Error: 
 Accuracy: 70.3%, Avg loss: 0.633935 

Epoch 2
-------------------------------


3380it [00:21, 160.36it/s]                          


Test Error: 
 Accuracy: 70.3%, Avg loss: 0.632216 

Epoch 3
-------------------------------


...

Test Error: 
 Accuracy: 74.9%, Avg loss: 0.552545 

Epoch 19
-------------------------------


3380it [00:21, 156.84it/s]                          


Test Error: 
 Accuracy: 76.6%, Avg loss: 0.506231 

Epoch 20
-------------------------------


3380it [00:22, 152.67it/s]                          


Test Error: 
 Accuracy: 77.4%, Avg loss: 0.496496 

